## 1. Setup

In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
import numpy as np
import tensorflow as tf
import keras
print(tf.__version__)

2.4.1


## 2. Custom Model

reference: [keras.Model](https://keras.io/api/models/model/)

In [45]:
class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]
        
    def call(self, inputs):
        x = inputs
        for layers in self.dense_layers:
            x = layers(x)
        return x
    
    def get_config(self):
        return {"hidden_units ": self.hidden_units}
    
    def from_config(cls, config):
        return cls(**config)
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
    ''' 
     def from_config(cls, config):
      This method is the reverse of get_config,
      capable of instantiating the same layer from the config
      dictionary. It does not handle layer connectivity
      (handled by Container), nor weights (handled by `set_weights`).
      # Arguments
          config: A Python dictionary, typically the
              output of get_config.
      
      return cls(**config)
    '''

In [47]:
model = CustomModel([16, 16, 10])
model.get_config()

{'hidden_units ': ListWrapper([16, 16, 10])}

In [42]:
#inputs = tf.keras.Input(shape=(3,))
#model.call(inputs)

In [7]:
## Build the model by calling it
input_arr = tf.random.uniform((1,5))
outputs = model(input_arr)
model.save("my_model_2")

INFO:tensorflow:Assets written to: my_model_2/assets


In [13]:
input_arr

<tf.Tensor: shape=(1, 5), dtype=float32, numpy=
array([[0.85459626, 0.27050006, 0.31505454, 0.77730906, 0.12544918]],
      dtype=float32)>

In [15]:
outputs

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[-0.9674891 , -0.05207668,  0.20079112, -0.11602056, -0.29946184,
         0.50982976,  1.2122071 ,  0.1363339 , -0.25818843,  0.8980546 ]],
      dtype=float32)>

## 3. Load the model

### 3.1 Load with the CustomModel argument 

In [8]:
loaded_1 = keras.models.load_model(
    "my_model_2", custom_objects={"CustomModel": CustomModel}
)

### 3.2 Load without the CustomModel Class

In [9]:
del CustomModel

In [10]:
loaded_2 = keras.models.load_model("my_model_2")


In [11]:
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

In [12]:
print("--> Original model:", model)
print("--> Model Loaded with custom objects:", loaded_1)
print("--> Model loaded without the custom object class:", loaded_2)


--> Original model: <__main__.CustomModel object at 0x7f578def9c70>
--> Model Loaded with custom objects: <__main__.CustomModel object at 0x7f578f1c2640>
--> Model loaded without the custom object class: <tensorflow.python.keras.saving.saved_model.load.CustomModel object at 0x7f578f1d1cd0>


### Note
The first loaded model is loaded using the config and CustomModel class. The second model is loaded by dynamically creating the model class that acts like the original model.

